In [ ]:
import kagglehub
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt


path = kagglehub.dataset_download("nirmalsankalana/rice-leaf-disease-image")
class_names = os.listdir(path)

In [ ]:
images = []
labels = []

for class_name in class_names:
    class_folder = os.path.join(path, class_name)
    
    for filename in os.listdir(class_folder):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            img_path = os.path.join(class_folder, filename)
            
            img = Image.open(img_path)
            img = img.convert('RGB')
            img = img.resize((128, 128))
            img_array = np.array(img)
            
            images.append(img_array)
            labels.append(class_name)

images_array = np.array(images)
labels_array = np.array(labels)

print("Images array shape:", images_array.shape)
print("Labels array shape:", labels_array.shape)